# Optimized Interactive ConceptNet Visualization

This notebook demonstrates a high-performance visualization of ConceptNet semantic relationships. We'll use our optimized preprocessing pipeline to load data efficiently and render interactive visualizations.

In [1]:
# Import necessary libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings
warnings.filterwarnings('ignore')

# Add project modules
sys.path.append('../Py Scripts')
from conceptnet_processor_v3 import ConceptNetStreamProcessor

# Try to import GPU libraries, but continue if not available
GPU_AVAILABLE = False
try:
    import cudf
    import cugraph
    import cupy as cp
    from pycuda import driver
    import pycuda.autoinit
    print(f"GPU: {driver.Device(0).name()}")
    print(f"CUDA Version: {driver.get_version()}")
    GPU_AVAILABLE = True
except ImportError:
    print("GPU libraries not available. Using CPU processing instead.")
except Exception as e:
    print(f"Error initializing GPU: {e}. Using CPU processing instead.")

# Interactive visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

GPU libraries not available. Using CPU processing instead.


In [2]:
def check_gpu_status():
    """Comprehensive GPU diagnostic function"""
    print("=== GPU Diagnostics ===")
    
    # Check CUDA availability via different libraries
    print("\n-- CUDA Status --")
    try:
        import torch
        print(f"PyTorch CUDA available: {torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"PyTorch: {torch.cuda.get_device_name(0)}")
            print(f"Memory allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
            print(f"Memory reserved: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")
    except ImportError:
        print("PyTorch not available")
    
    try:
        import pycuda.driver as cuda
        cuda.init()
        print(f"PyCUDA device count: {cuda.Device.count()}")
        for i in range(cuda.Device.count()):
            device = cuda.Device(i)
            props = device.get_attributes()
            print(f"  Device {i}: {device.name()}")
            print(f"    Memory: {device.total_memory() / 1e9:.2f} GB")
            print(f"    Compute Capability: {props[cuda.device_attribute.COMPUTE_CAPABILITY_MAJOR]}.{props[cuda.device_attribute.COMPUTE_CAPABILITY_MINOR]}")
    except Exception as e:
        print(f"PyCUDA error: {e}")
    
    try:
        import cudf
        print("\n-- RAPIDS Status --")
        print(f"cuDF version: {cudf.__version__}")
        # Test with simple dataframe operation
        test_df = cudf.DataFrame({'a': [1, 2, 3]})
        print(f"cuDF test: {test_df.a.sum()} (should be 6)")
    except Exception as e:
        print(f"RAPIDS error: {e}")

# Call this at the beginning of your notebook
check_gpu_status()

=== GPU Diagnostics ===

-- CUDA Status --
PyTorch not available
PyCUDA error: No module named 'pycuda'
RAPIDS error: No module named 'cudf'


## 1. Optimized Data Loading
We'll use our efficient data processor to load a manageable sample of ConceptNet data.

In [9]:
# Initialize our optimized processor
processor = ConceptNetStreamProcessor(
    input_dir='../Data/Input',
    output_dir='../Data/Processed'
)

# Load a small sample for testing visualization
# For initial testing, we'll use a very small fraction
sample_size = 1  # Start with 1% for speed
max_concepts = 500  # Limit number of unique concepts
min_weight = 1.5     # Higher threshold for stronger relationships

# Load English data
english_df = processor.load_processed_data(
    lang='en', 
    sample_size=sample_size, 
    max_rows=200_000
)

# Load German data
german_df = processor.load_processed_data(
    lang='de', 
    sample_size=sample_size,
    max_rows=200_000
)

# Display statistics
print(f"Loaded {len(english_df)} English assertions with {english_df['clean_start'].nunique()} source concepts")
print(f"Loaded {len(german_df)} German assertions with {german_df['clean_start'].nunique()} source concepts")

# Filter by weight for better visualization
english_df = english_df[english_df['weight'] >= min_weight]
german_df = german_df[german_df['weight'] >= min_weight]

print(f"After filtering: {len(english_df)} English and {len(german_df)} German assertions")

Dask loading failed with error: 'DataFrame' object has no attribute 'compute'
Falling back to pandas for data loading
Dask loading failed with error: 'DataFrame' object has no attribute 'compute'
Falling back to pandas for data loading
Loaded 200000 English assertions with 32548 source concepts
Loaded 200000 German assertions with 79021 source concepts
After filtering: 17468 English and 3085 German assertions


## 2. Graph Construction
Now we'll construct semantic graphs from our data.

In [10]:
def build_graph(df, weight_col='weight'):
    """Build a graph from ConceptNet dataframe"""
    # Create NetworkX graph
    G = nx.DiGraph()
    
    # Add edges with attributes
    for _, row in tqdm(df.iterrows(), total=len(df)):
        G.add_edge(
            row['clean_start'], 
            row['clean_end'], 
            weight=row[weight_col],
            relation=row['relation']
        )
    return G

# Build graphs for both languages
print("Building English semantic graph...")
english_graph = build_graph(english_df)

print("Building German semantic graph...")
german_graph = build_graph(german_df)

Building English semantic graph...


100%|██████████| 17468/17468 [00:00<00:00, 69878.33it/s]


Building German semantic graph...


100%|██████████| 3085/3085 [00:00<00:00, 63922.40it/s]


## 3. Graph Layout Computation
Calculate optimal positions for nodes using force-directed layout algorithms.

In [11]:
def compute_layout(graph, iterations=500):
    """Compute force-directed layout for the graph"""
    # Compute layout using NetworkX
    print(f"Computing layout for graph with {len(graph.nodes())} nodes and {len(graph.edges())} edges...")
    pos = nx.spring_layout(graph, k=0.15, iterations=iterations, seed=42)
    
    # Convert to DataFrame for easier processing later
    layout_df = pd.DataFrame({
        'node': list(pos.keys()),
        'x': [pos[node][0] for node in pos],
        'y': [pos[node][1] for node in pos]
    })
    
    return layout_df

# Compute layouts
print("Computing English semantic layout...")
english_layout = compute_layout(english_graph)

print("Computing German semantic layout...")
german_layout = compute_layout(german_graph)

Computing English semantic layout...
Computing layout for graph with 4809 nodes and 4467 edges...
Computing German semantic layout...
Computing layout for graph with 3044 nodes and 2997 edges...


## 4. Interactive Visualization
Create an interactive visualization for exploring the semantic networks.

In [12]:
class SemanticVisualizer:
    """Interactive semantic network visualizer"""
    
    def __init__(self, graph, layout, title="Semantic Network", height=800, width=1200):
        self.graph = graph
        self.layout = layout
        self.title = title
        self.height = height
        self.width = width
        self.node_size_scale = 10
        self.edge_width_scale = 0.5
        self.selected_nodes = set()
        
        # Precompute node metrics for sizing
        self._compute_node_metrics()
        
    def _compute_node_metrics(self):
        """Compute node metrics for sizing and coloring"""
        # Calculate node degrees
        self.node_degrees = dict(self.graph.degree())
        
        # Calculate centrality (use approximate betweenness for large graphs)
        if len(self.graph) > 1000:
            print("Computing approximate betweenness centrality...")
            self.node_betweenness = nx.betweenness_centrality(self.graph, k=min(100, len(self.graph)))
        else:
            print("Computing exact betweenness centrality...")
            self.node_betweenness = nx.betweenness_centrality(self.graph)
        
        # Create node dataframe
        nodes = list(self.graph.nodes())
        self.node_df = pd.DataFrame({
            'node': nodes,
            'degree': [self.node_degrees.get(n, 0) for n in nodes],
            'betweenness': [self.node_betweenness.get(n, 0) for n in nodes]
        })
        
        # Merge with layout
        self.node_layout_df = pd.merge(self.layout, self.node_df, on='node', how='inner')
        
        # Create edges dataframe
        edges = list(self.graph.edges(data=True))
        self.edge_df = pd.DataFrame({
            'source': [e[0] for e in edges],
            'target': [e[1] for e in edges],
            'weight': [e[2].get('weight', 1.0) for e in edges],
            'relation': [e[2].get('relation', 'related') for e in edges]
        })
    
    def render(self, show_labels=True, max_edges=5000, full_screen=False):
        """Render the interactive semantic graph visualization
        
        Args:
            show_labels (bool): Whether to show node labels
            max_edges (int): Maximum number of edges to display
            full_screen (bool): Whether to render in full screen mode
        """
        # Set dimensions based on display mode
        if full_screen:
            height = "100vh"  # 100% of viewport height
            width = "100%"    # 100% of viewport width
        else:
            height = self.height
            width = self.width
            
        # Limit number of edges for performance
        if len(self.edge_df) > max_edges:
            # Select edges with highest weights
            edge_sample = self.edge_df.sort_values('weight', ascending=False).head(max_edges)
        else:
            edge_sample = self.edge_df
            
        # Create edge traces
        edge_trace = []
        
        # Get coordinates for each edge
        for _, edge in tqdm(edge_sample.iterrows(), total=len(edge_sample), desc="Creating edge traces"):
            source = edge['source']
            target = edge['target']
            relation = edge['relation']
            weight = edge['weight']
            
            # Get node positions
            source_pos = self.node_layout_df[self.node_layout_df['node'] == source]
            target_pos = self.node_layout_df[self.node_layout_df['node'] == target]
            
            if len(source_pos) == 0 or len(target_pos) == 0:
                continue
                
            x0, y0 = source_pos['x'].values[0], source_pos['y'].values[0]
            x1, y1 = target_pos['x'].values[0], target_pos['y'].values[0]
            
            trace = go.Scatter(
                x=[x0, x1, None],
                y=[y0, y1, None],
                mode='lines',
                line=dict(
                    width=weight * self.edge_width_scale,
                    color='rgba(150,150,150,0.3)'
                ),
                hoverinfo='none',
                showlegend=False
            )
            edge_trace.append(trace)
        
        # Create node trace
        node_trace = go.Scatter(
            x=self.node_layout_df['x'],
            y=self.node_layout_df['y'],
            mode='markers+text' if show_labels else 'markers',
            text=self.node_layout_df['node'] if show_labels else None,
            textposition='top center',
            textfont=dict(size=10, color='black'),
            marker=dict(
                size=self.node_layout_df['degree'] * self.node_size_scale,
                color=self.node_layout_df['betweenness'],
                colorscale='Viridis',
                line=dict(width=1, color='black'),
                opacity=0.8,
                showscale=True,
                colorbar=dict(title='Centrality')
            ),
            hovertemplate=(
                '<b>%{text}</b><br>'
                'Degree: %{marker.size:.1f}<br>'
                'Centrality: %{marker.color:.4f}'
                '<extra></extra>'
            )
        )
        
        # Combine all traces
        all_traces = edge_trace + [node_trace]
        
        # Create layout
        layout = go.Layout(
            title=dict(
                text=self.title,
                font=dict(size=24)
            ),
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20, l=5, r=5, t=40),
            height=height,
            width=width,
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            plot_bgcolor='rgba(240,240,240,1)',
            dragmode='pan',
            updatemenus=[
                dict(
                    type='buttons',
                    showactive=False,
                    buttons=[
                        dict(
                            label='Reset View',
                            method='relayout',
                            args=['xaxis.range', [None, None]]
                        )
                    ]
                )
            ]
        )
        
        # Create figure
        fig = go.Figure(data=all_traces, layout=layout)
        
        # Add zoom capability
        fig.update_layout(
            newshape=dict(line_color='#ff0000'),
            modebar=dict(
                add=['drawrect', 'eraseshape', 'zoomIn', 'zoomOut', 'autoScale', 'resetScale']
            )
        )
        
        # For full screen, open in new browser tab
        if full_screen:
            import tempfile
            import webbrowser
            import os
            
            # Create a temporary HTML file
            temp_dir = tempfile.gettempdir()
            temp_path = os.path.join(temp_dir, f"semantic_viz_{hash(self.title)}.html")
            
            # Write the figure to the temporary file with full screen settings
            fig.write_html(
                temp_path,
                full_html=True,
                include_plotlyjs='cdn',
                config={
                    'displayModeBar': True,
                    'scrollZoom': True,
                    'responsive': True
                }
            )
            
            # Open the file in the default browser
            webbrowser.open('file://' + temp_path, new=2)
            
            # Return a message
            return HTML(f"<p>Visualization opened in a new browser tab. If it didn't open automatically, <a href='file://{temp_path}' target='_blank'>click here</a>.</p>")
        
        return fig

# Create the visualizers
english_vis = SemanticVisualizer(
    english_graph, 
    english_layout, 
    title="English Semantic Network",
    height=900,
    width=1200
)

german_vis = SemanticVisualizer(
    german_graph, 
    german_layout, 
    title="German Semantic Network",
    height=900,
    width=1200
)

Computing approximate betweenness centrality...
Computing approximate betweenness centrality...


## 5. Display the Interactive Visualizations
Let's render our visualizations and explore the semantic networks interactively.

In [ ]:
# Render English visualization
english_fig = english_vis.render(show_labels=True, max_edges=3000)
english_fig.show()

In [ ]:
# Render German visualization
german_fig = german_vis.render(show_labels=True, max_edges=3000)
german_fig.show()

## 6. Interactive Multi-Language Semantic Explorer
Let's create an interactive tool that allows for exploring cross-language semantic relationships.

In [23]:
class SemanticExplorer:
    """Interactive multi-language semantic explorer"""
    
    def __init__(self, en_graph, en_layout, de_graph, de_layout):
        self.en_graph = en_graph
        self.en_layout = en_layout
        self.de_graph = de_graph
        self.de_layout = de_layout
        
        # Create visualizers
        self.en_vis = SemanticVisualizer(en_graph, en_layout, title="English Semantic Network")
        self.de_vis = SemanticVisualizer(de_graph, de_layout, title="German Semantic Network")
        
        # Create UI elements
        self._setup_ui()
        
    def _setup_ui(self):
        """Set up interactive UI elements"""
        self.search_input = widgets.Text(
            value='',
            placeholder='Search for a concept',
            description='Search:',
            disabled=False
        )
        
        self.language_selector = widgets.RadioButtons(
            options=['English', 'German', 'Both'],
            value='Both',
            description='Language:',
            disabled=False
        )
        
        self.depth_slider = widgets.IntSlider(
            value=1,
            min=1,
            max=3,
            step=1,
            description='Network Depth:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True
        )
        
        self.min_weight_slider = widgets.FloatSlider(
            value=1.5,
            min=1.0,
            max=5.0,
            step=0.1,
            description='Min Weight:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True
        )
        
        self.show_labels_checkbox = widgets.Checkbox(
            value=True,
            description='Show Labels',
            disabled=False
        )
        
        self.fullscreen_checkbox = widgets.Checkbox(
            value=False,
            description='Open in Full Screen',
            disabled=False
        )
        
        self.search_button = widgets.Button(
            description='Explore',
            disabled=False,
            button_style='primary',
            tooltip='Click to explore the semantic network'
        )
        
        self.output = widgets.Output()
        
        # Wire up events
        self.search_button.on_click(self._on_search_clicked)
        
    def show(self):
        """Display the interactive explorer"""
        # Create layout
        top_controls = widgets.HBox([self.search_input, self.language_selector])
        bottom_controls = widgets.HBox([
            self.depth_slider, 
            self.min_weight_slider, 
            self.show_labels_checkbox, 
            self.fullscreen_checkbox,
            self.search_button
        ])
        
        # Display the UI
        display(widgets.VBox([top_controls, bottom_controls, self.output]))
        
    def _on_search_clicked(self, b):
        """Handle search button click"""
        with self.output:
            clear_output()
            concept = self.search_input.value.lower().strip()
            
            if not concept:
                print("Please enter a concept to search for.")
                return
                
            print(f"Exploring concept: {concept}")
            
            # Get parameters
            language = self.language_selector.value
            depth = self.depth_slider.value
            min_weight = self.min_weight_slider.value
            show_labels = self.show_labels_checkbox.value
            full_screen = self.fullscreen_checkbox.value
            
            # Generate visualizations based on selection
            if language in ['English', 'Both']:
                print("Generating English visualization...")
                en_fig = self._generate_subgraph_viz(
                    concept, 
                    self.en_vis, 
                    depth, 
                    min_weight, 
                    show_labels,
                    full_screen,
                    title=f"English Semantic Network for '{concept}'"
                )
                if en_fig:
                    display(en_fig)
                else:
                    print(f"Concept '{concept}' not found in English network.")
            
            if language in ['German', 'Both'] and not (full_screen and language == 'Both'):
                print("Generating German visualization...")
                de_fig = self._generate_subgraph_viz(
                    concept, 
                    self.de_vis, 
                    depth, 
                    min_weight, 
                    show_labels,
                    full_screen,
                    title=f"German Semantic Network for '{concept}'"
                )
                if de_fig:
                    display(de_fig)
                else:
                    print(f"Concept '{concept}' not found in German network.")
    
    def _generate_subgraph_viz(self, concept, visualizer, depth, min_weight, show_labels, full_screen=False, title=""):
        """Generate a subgraph visualization for a concept"""
        # Check if concept exists in the graph nodes
        if concept not in visualizer.graph.nodes():
            return None
            
        # Extract subgraph using BFS
        nodes_to_include = {concept}
        frontier = {concept}
        
        # BFS to depth
        for _ in range(depth):
            new_frontier = set()
            for node in frontier:
                neighbors = set(n for n in visualizer.graph.neighbors(node))
                new_frontier.update(neighbors)
            frontier = new_frontier - nodes_to_include
            nodes_to_include.update(frontier)
        
        # Create subgraph
        subgraph = visualizer.graph.subgraph(nodes_to_include)
        
        # Filter edges by weight
        edges_to_remove = []
        for u, v, data in subgraph.edges(data=True):
            if data.get('weight', 0) < min_weight:
                edges_to_remove.append((u, v))
        
        # Remove edges below weight threshold
        for edge in edges_to_remove:
            subgraph.remove_edge(*edge)
        
        # Extract layout for subgraph
        nodes_in_subgraph = set(subgraph.nodes())
        sub_layout = visualizer.layout[visualizer.layout['node'].isin(nodes_in_subgraph)].copy()
        
        # Create visualization
        sub_vis = SemanticVisualizer(subgraph, sub_layout, title=title)
        
        # Ensure node metrics are computed
        sub_vis._compute_node_metrics()
        
        # Render with full screen option if requested
        return sub_vis.render(show_labels=show_labels, full_screen=full_screen)

# Create the explorer
explorer = SemanticExplorer(english_graph, english_layout, german_graph, german_layout)
explorer.show()

Computing approximate betweenness centrality...
Computing approximate betweenness centrality...


## 7. Semantic Path Finder

Let's implement a simple semantic path finder that can discover relationships between concepts. This is useful for understanding how different concepts relate to each other through intermediate relationships.

In [ ]:
class SemanticPathFinder:
    """Find and visualize semantic paths between concepts"""
    
    def __init__(self, graph, max_suggestions=100):
        self.graph = graph
        self.max_suggestions = max_suggestions
        self._setup_ui()
    
    def _setup_ui(self):
        """Set up the path finder UI with better concept selection"""
        # Create a sorted list of all concepts (nodes) in the graph
        all_concepts = sorted(list(self.graph.nodes()))
        self.top_concepts = self._get_top_concepts(20)  # Get 20 most connected concepts
        
        # Source concept search box with auto-complete
        self.source_search = widgets.Text(
            value='',
            placeholder='Search for source concept',
            description='Source:',
            disabled=False
        )
        
        # Target concept search box with auto-complete
        self.target_search = widgets.Text(
            value='',
            placeholder='Search for target concept',
            description='Target:',
            disabled=False
        )
        
        # Dropdown for source selection (will be populated based on search)
        self.source_dropdown = widgets.Dropdown(
            options=self.top_concepts,
            description='Select:',
            disabled=False
        )
        
        # Dropdown for target selection (will be populated based on search)
        self.target_dropdown = widgets.Dropdown(
            options=self.top_concepts,
            description='Select:',
            disabled=False
        )
        
        # Max path length slider
        self.max_path_length = widgets.IntSlider(
            value=3,
            min=1,
            max=5,
            step=1,
            description='Max Path Length:',
            disabled=False,
            continuous_update=False
        )
        
        # Find paths button
        self.find_button = widgets.Button(
            description='Find Paths',
            disabled=False,
            button_style='primary'
        )
        
        # Output area
        self.output = widgets.Output()
        
        # Connect events
        self.find_button.on_click(self._on_find_clicked)
        self.source_search.observe(self._on_source_search_change, names='value')
        self.target_search.observe(self._on_target_search_change, names='value')
    
    def _get_top_concepts(self, n):
        """Get the top n most connected concepts in the graph"""
        # Get concepts with most connections
        degree_dict = dict(self.graph.degree())
        sorted_concepts = sorted(degree_dict.items(), key=lambda x: x[1], reverse=True)
        return [concept for concept, _ in sorted_concepts[:n]]
    
    def _on_source_search_change(self, change):
        """Filter source concept options based on search term"""
        search_term = change['new'].lower().strip()
        if not search_term:
            # If search is empty, show top concepts
            self.source_dropdown.options = self.top_concepts
        else:
            # Find matching concepts
            matches = [n for n in self.graph.nodes() if search_term in n.lower()]
            # Limit matches to avoid overloading the dropdown
            matches = sorted(matches)[:self.max_suggestions]
            if matches:
                self.source_dropdown.options = matches
            else:
                self.source_dropdown.options = ['No matches found']
    
    def _on_target_search_change(self, change):
        """Filter target concept options based on search term"""
        search_term = change['new'].lower().strip()
        if not search_term:
            # If search is empty, show top concepts
            self.target_dropdown.options = self.top_concepts
        else:
            # Find matching concepts
            matches = [n for n in self.graph.nodes() if search_term in n.lower()]
            # Limit matches to avoid overloading the dropdown
            matches = sorted(matches)[:self.max_suggestions]
            if matches:
                self.target_dropdown.options = matches
            else:
                self.target_dropdown.options = ['No matches found']
    
    def show(self):
        """Display the path finder interface"""
        source_box = widgets.VBox([
            widgets.Label('Source Concept:'),
            self.source_search,
            self.source_dropdown
        ])
        
        target_box = widgets.VBox([
            widgets.Label('Target Concept:'),
            self.target_search,
            self.target_dropdown
        ])
        
        display(widgets.VBox([
            widgets.HBox([source_box, target_box, self.max_path_length]),
            self.find_button,
            self.output
        ]))
    
    def _on_find_clicked(self, b):
        """Handle button click event"""
        with self.output:
            clear_output()
            
            source = self.source_dropdown.value
            target = self.target_dropdown.value
            max_length = self.max_path_length.value
            
            if source == 'No matches found' or target == 'No matches found':
                print("Please select valid source and target concepts.")
                return
            
            if not source or not target:
                print("Please select both source and target concepts.")
                return
            
            # Check if both concepts exist in the graph
            if source not in self.graph:
                print(f"Source concept '{source}' not found in the graph.")
                return
                
            if target not in self.graph:
                print(f"Target concept '{target}' not found in the graph.")
                return
            
            print(f"Finding paths from '{source}' to '{target}' (max length: {max_length})...")
            paths = self._find_all_paths(source, target, max_length)
            
            if not paths:
                print(f"No paths found between '{source}' and '{target}' within {max_length} steps.")
            else:
                print(f"Found {len(paths)} path(s):")
                self._display_paths(paths)
    
    def _find_all_paths(self, source, target, max_length):
        """Find all paths between source and target up to max_length"""
        # Use limited depth-first search to find paths
        paths = []
        self._dfs(source, target, max_length, [source], paths, set([source]))
        return paths
    
    def _dfs(self, current, target, depth_left, current_path, result_paths, visited):
        # Base case: reached target
        if current == target:
            result_paths.append(current_path.copy())
            return
            
        # Base case: max depth reached
        if depth_left <= 0:
            return
            
        # Explore neighbors
        for neighbor in self.graph.neighbors(current):
            if neighbor not in visited:
                # Add to current path and visited set
                current_path.append(neighbor)
                visited.add(neighbor)
                
                # Recursively search from this neighbor
                self._dfs(neighbor, target, depth_left - 1, current_path, result_paths, visited)
                
                # Backtrack
                current_path.pop()
                visited.remove(neighbor)
    
    def _display_paths(self, paths):
        """Display found paths with relationship information"""
        for i, path in enumerate(paths):
            print(f"\nPath {i+1}: {' → '.join(path)}")
            print("Relationships:")
            
            # Show the relationships between each pair of concepts in the path
            for j in range(len(path) - 1):
                source, target = path[j], path[j+1]
                if self.graph.has_edge(source, target):
                    edge_data = self.graph.get_edge_data(source, target)
                    relation = edge_data.get('relation', 'related')
                    weight = edge_data.get('weight', 1.0)
                    print(f"  {source} --[{relation} ({weight:.2f})]-> {target}")
                else:
                    print(f"  {source} --[?]-> {target} (relationship data missing)")
            
            # Add a visual separator between paths
            if i < len(paths) - 1:
                print("---")
        
        # Visualize the first path if available
        if paths:
            self._visualize_path(paths[0])
    
    def _visualize_path(self, path):
        """Create a simple visualization of the path"""
        # Create a subgraph containing just the path
        path_graph = nx.DiGraph()
        
        # Add nodes and edges from the path
        for i in range(len(path) - 1):
            source, target = path[i], path[i+1]
            
            # Get edge data if available
            if self.graph.has_edge(source, target):
                edge_data = self.graph.get_edge_data(source, target)
                path_graph.add_edge(source, target, **edge_data)
            else:
                path_graph.add_edge(source, target)
        
        # Create a simple visualization
        plt.figure(figsize=(10, 4))
        pos = nx.spring_layout(path_graph)
        
        # Draw the path
        nx.draw(path_graph, pos, with_labels=True, node_color='skyblue', 
                node_size=2000, edge_color='gray', width=2, font_weight='bold')
        
        # Add edge labels
        edge_labels = {}
        for source, target, data in path_graph.edges(data=True):
            relation = data.get('relation', '?')
            edge_labels[(source, target)] = relation
        
        nx.draw_networkx_edge_labels(path_graph, pos, edge_labels=edge_labels)
        
        plt.title(f"Semantic Path: {' → '.join(path)}")
        plt.axis('off')
        plt.tight_layout()
        plt.show()

# Create path finders for both languages with improved UI
print("English Semantic Path Finder:")
english_path_finder = SemanticPathFinder(english_graph)
english_path_finder.show()

print("\nGerman Semantic Path Finder:")
german_path_finder = SemanticPathFinder(german_graph)
german_path_finder.show()

English Semantic Path Finder:



German Semantic Path Finder:


## 8. Cross-Language Semantic Mapper

Let's build a tool that can map concepts across languages (English to German) and visualize the semantic bridges between them.

In [29]:
class CrossLanguageMapper:
    """Maps semantic concepts across languages and visualizes the connections"""
    
    def __init__(self, en_graph, de_graph, max_suggestions=100):
        self.en_graph = en_graph
        self.de_graph = de_graph
        self.max_suggestions = max_suggestions
        self._setup_ui()
    
    def _setup_ui(self):
        """Set up interactive UI for cross-language mapping"""
        # Get top concepts from both languages
        self.top_en_concepts = self._get_top_concepts(self.en_graph, 20)
        self.top_de_concepts = self._get_top_concepts(self.de_graph, 20)
        
        # Source concept (English) selection
        self.en_search = widgets.Text(
            value='',
            placeholder='Search for English concept',
            description='English:',
            disabled=False
        )
        self.en_dropdown = widgets.Dropdown(
            options=self.top_en_concepts,
            description='Select:',
            disabled=False
        )
        
        # Target concept (German) for comparison
        self.de_search = widgets.Text(
            value='',
            placeholder='Search for German concept',
            description='German:',
            disabled=False
        )
        self.de_dropdown = widgets.Dropdown(
            options=self.top_de_concepts,
            description='Select:',
            disabled=False
        )
        
        # Mapping parameters
        self.semantic_depth = widgets.IntSlider(
            value=2,
            min=1,
            max=4,
            step=1,
            description='Semantic Depth:',
            disabled=False,
            continuous_update=False
        )
        
        self.similarity_threshold = widgets.FloatSlider(
            value=0.4,
            min=0.1,
            max=0.9,
            step=0.1,
            description='Similarity Threshold:',
            disabled=False,
            continuous_update=False
        )
        
        # Action button
        self.map_button = widgets.Button(
            description='Map Concepts',
            disabled=False,
            button_style='primary'
        )
        
        # Output area
        self.output = widgets.Output()
        
        # Connect events
        self.map_button.on_click(self._on_map_clicked)
        self.en_search.observe(self._on_en_search_change, names='value')
        self.de_search.observe(self._on_de_search_change, names='value')
    
    def _get_top_concepts(self, graph, n):
        """Get top n concepts with highest degree from the graph"""
        degree_dict = dict(graph.degree())
        sorted_concepts = sorted(degree_dict.items(), key=lambda x: x[1], reverse=True)
        return [concept for concept, _ in sorted_concepts[:n]]
    
    def _on_en_search_change(self, change):
        """Update English concept dropdown based on search"""
        search_term = change['new'].lower().strip()
        if not search_term:
            self.en_dropdown.options = self.top_en_concepts
        else:
            matches = [n for n in self.en_graph.nodes() if search_term in n.lower()]
            matches = sorted(matches)[:self.max_suggestions]
            if matches:
                self.en_dropdown.options = matches
            else:
                self.en_dropdown.options = ['No matches found']
    
    def _on_de_search_change(self, change):
        """Update German concept dropdown based on search"""
        search_term = change['new'].lower().strip()
        if not search_term:
            self.de_dropdown.options = self.top_de_concepts
        else:
            matches = [n for n in self.de_graph.nodes() if search_term in n.lower()]
            matches = sorted(matches)[:self.max_suggestions]
            if matches:
                self.de_dropdown.options = matches
            else:
                self.de_dropdown.options = ['No matches found']
    
    def show(self):
        """Display the cross-language mapper interface"""
        en_box = widgets.VBox([
            widgets.Label('English Concept:'),
            self.en_search,
            self.en_dropdown
        ])
        
        de_box = widgets.VBox([
            widgets.Label('German Concept:'),
            self.de_search,
            self.de_dropdown
        ])
        
        controls = widgets.HBox([
            en_box,
            de_box,
            widgets.VBox([
                self.semantic_depth,
                self.similarity_threshold
            ])
        ])
        
        display(widgets.VBox([
            controls,
            self.map_button,
            self.output
        ]))
    
    def _on_map_clicked(self, b):
        """Handle mapping button click"""
        with self.output:
            clear_output()
            
            en_concept = self.en_dropdown.value
            de_concept = self.de_dropdown.value
            
            # Validate inputs
            if en_concept in ['No matches found'] or de_concept in ['No matches found']:
                print("Please select valid concepts in both languages.")
                return
                
            if not en_concept or not en_concept in self.en_graph:
                print(f"English concept '{en_concept}' not found in the graph.")
                return
                
            if not de_concept or not de_concept in self.de_graph:
                print(f"German concept '{de_concept}' not found in the graph.")
                return
            
            depth = self.semantic_depth.value
            threshold = self.similarity_threshold.value
            
            print(f"Mapping {en_concept} (EN) to {de_concept} (DE) with depth {depth}...")
            self._create_cross_language_mapping(en_concept, de_concept, depth, threshold)
    
    def _find_neighborhood(self, graph, concept, depth):
        """Extract neighborhood of concepts up to specified depth"""
        neighborhood = {concept}
        frontier = {concept}
        
        # BFS to collect neighbors
        for _ in range(depth):
            new_frontier = set()
            for node in frontier:
                # Add both outgoing and incoming neighbors
                out_neighbors = set(graph.neighbors(node))
                in_neighbors = set(graph.predecessors(node))
                new_frontier.update(out_neighbors | in_neighbors)
            
            # Remove already processed nodes
            frontier = new_frontier - neighborhood
            neighborhood.update(frontier)
        
        return neighborhood
    
    def _compute_similarity(self, concept1, concept2):
        """Compute simple string similarity between concepts"""
        # Convert to lowercase for comparison
        c1 = concept1.lower()
        c2 = concept2.lower()
        
        # Use Jaccard similarity on character trigrams
        def get_trigrams(s):
            return {s[i:i+3] for i in range(len(s)-2)} if len(s) > 2 else {s}
        
        trigrams1 = get_trigrams(c1)
        trigrams2 = get_trigrams(c2)
        
        # Calculate Jaccard similarity
        intersection = len(trigrams1.intersection(trigrams2))
        union = len(trigrams1.union(trigrams2))
        
        if union == 0:
            return 0
        return intersection / union
    
    def _create_semantic_connections(self, en_neighborhood, de_neighborhood, threshold):
        """Create connections between semantically similar concepts across languages"""
        connections = []
        
        # Find concept pairs that exceed similarity threshold
        for en_concept in en_neighborhood:
            for de_concept in de_neighborhood:
                similarity = self._compute_similarity(en_concept, de_concept)
                if similarity >= threshold:
                    connections.append((en_concept, de_concept, similarity))
        
        # Sort by similarity score
        return sorted(connections, key=lambda x: x[2], reverse=True)
    
    def _create_cross_language_mapping(self, en_concept, de_concept, depth, threshold):
        """Create and visualize cross-language semantic mapping"""
        # Extract neighborhoods in both languages
        print("Extracting semantic neighborhoods...")
        en_neighborhood = self._find_neighborhood(self.en_graph, en_concept, depth)
        de_neighborhood = self._find_neighborhood(self.de_graph, de_concept, depth)
        
        print(f"Found {len(en_neighborhood)} English concepts and {len(de_neighborhood)} German concepts")
        
        # Compute cross-language semantic connections
        print("Computing cross-language connections...")
        connections = self._create_semantic_connections(en_neighborhood, de_neighborhood, threshold)
        
        print(f"Found {len(connections)} cross-language connections above threshold {threshold}")
        
        # Display top connections
        self._display_connections(connections[:20])  # Show top 20 connections
        
        # Visualize the cross-language mapping
        self._visualize_mapping(en_concept, de_concept, en_neighborhood, de_neighborhood, connections)
    
    def _display_connections(self, connections):
        """Display top cross-language connections"""
        if not connections:
            print("No semantic connections found above threshold.")
            return
            
        print("\nTop Cross-Language Connections:")
        print("-" * 60)
        print(f"{'English Concept':<25} | {'German Concept':<25} | {'Similarity':<10}")
        print("-" * 60)
        
        for en_concept, de_concept, similarity in connections:
            print(f"{en_concept:<25} | {de_concept:<25} | {similarity:.2f}")
    
    def _visualize_mapping(self, en_root, de_root, en_neighborhood, de_neighborhood, connections):
        """Create an interactive visualization of cross-language mapping"""
        # Create a combined graph for visualization
        G = nx.DiGraph()
        
        # Add English nodes (blue)
        for node in en_neighborhood:
            G.add_node(node, language='en', color='blue')
            
        # Add German nodes (red)
        for node in de_neighborhood:
            # Add 'DE:' prefix to avoid node name collisions
            node_id = f"DE:{node}"
            G.add_node(node_id, language='de', name=node, color='red')
        
        # Add English graph connections
        for source in en_neighborhood:
            for target in self.en_graph.neighbors(source):
                if target in en_neighborhood:
                    if self.en_graph.has_edge(source, target):
                        data = self.en_graph.get_edge_data(source, target)
                        weight = data.get('weight', 1.0)
                        relation = data.get('relation', 'related')
                        G.add_edge(source, target, weight=weight, relation=relation, type='en-en')
        
        # Add German graph connections
        for source in de_neighborhood:
            source_id = f"DE:{source}"
            for target in self.de_graph.neighbors(source):
                if target in de_neighborhood:
                    target_id = f"DE:{target}"
                    if self.de_graph.has_edge(source, target):
                        data = self.de_graph.get_edge_data(source, target)
                        weight = data.get('weight', 1.0)
                        relation = data.get('relation', 'related')
                        G.add_edge(source_id, target_id, weight=weight, relation=relation, type='de-de')
        
        # Add cross-language connections
        for en_concept, de_concept, similarity in connections:
            de_id = f"DE:{de_concept}"
            G.add_edge(en_concept, de_id, weight=similarity, relation='similar', type='cross-lang')
        
        # Create positions using a spring layout, with English and German separated
        pos = nx.spring_layout(G, k=0.5, iterations=100, seed=42)
        
        # Adjust positions to separate languages visually
        for node, position in pos.items():
            if isinstance(node, str) and node.startswith('DE:'):
                pos[node] = (position[0] + 3, position[1]) # Shift German nodes to the right
        
        # Create the figure
        plt.figure(figsize=(20, 10))
        
        # Draw English nodes (blue)
        en_nodes = [n for n, attr in G.nodes(data=True) if attr.get('language') == 'en']
        nx.draw_networkx_nodes(G, pos, nodelist=en_nodes, node_color='skyblue', 
                           node_size=500, alpha=0.8)
        
        # Draw German nodes (red)
        de_nodes = [n for n, attr in G.nodes(data=True) if attr.get('language') == 'de']
        nx.draw_networkx_nodes(G, pos, nodelist=de_nodes, node_color='lightcoral', 
                           node_size=500, alpha=0.8)
        
        # Highlight root nodes (green)
        root_nodes = [en_root, f"DE:{de_root}"]
        nx.draw_networkx_nodes(G, pos, nodelist=root_nodes, node_color='lightgreen',
                           node_size=700, alpha=1.0)
        
        # Draw intra-language edges
        en_en_edges = [(u, v) for u, v, attr in G.edges(data=True) if attr.get('type') == 'en-en']
        de_de_edges = [(u, v) for u, v, attr in G.edges(data=True) if attr.get('type') == 'de-de']
        
        nx.draw_networkx_edges(G, pos, edgelist=en_en_edges, edge_color='blue', alpha=0.3, width=1)
        nx.draw_networkx_edges(G, pos, edgelist=de_de_edges, edge_color='red', alpha=0.3, width=1)
        
        # Draw cross-language edges
        cross_lang_edges = [(u, v) for u, v, attr in G.edges(data=True) if attr.get('type') == 'cross-lang']
        nx.draw_networkx_edges(G, pos, edgelist=cross_lang_edges, edge_color='purple', 
                           style='dashed', alpha=0.7, width=1.5)
        
        # Add labels for both languages
        en_labels = {n: n for n in en_nodes}
        de_labels = {n: G.nodes[n]['name'] for n in de_nodes}  # Use the original name without DE: prefix
        
        nx.draw_networkx_labels(G, pos, labels=en_labels, font_size=9, font_color='black')
        nx.draw_networkx_labels(G, pos, labels=de_labels, font_size=9, font_color='black')
        
        # Add title and legend
        plt.title(f"Cross-Language Semantic Mapping: '{en_root}' (EN) ↔ '{de_root}' (DE)", fontsize=16)
        
        # Legend elements
        from matplotlib.lines import Line2D
        from matplotlib.patches import Patch
        
        legend_elements = [
            Patch(facecolor='skyblue', edgecolor='black', alpha=0.8, label='English Concepts'),
            Patch(facecolor='lightcoral', edgecolor='black', alpha=0.8, label='German Concepts'),
            Patch(facecolor='lightgreen', edgecolor='black', alpha=1.0, label='Root Concepts'),
            Line2D([0], [0], color='blue', alpha=0.3, label='English Connections'),
            Line2D([0], [0], color='red', alpha=0.3, label='German Connections'),
            Line2D([0], [0], color='purple', linestyle='dashed', alpha=0.7, label='Cross-Language Similarities')
        ]
        
        plt.legend(handles=legend_elements, loc='upper right')
        
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        
        # Also create an interactive Plotly visualization
        self._create_interactive_visualization(G, pos, en_root, de_root)
    
    def _create_interactive_visualization(self, G, pos, en_root, de_root):
        """Create an interactive visualization using Plotly"""
        # Prepare node traces
        en_nodes = [n for n, attr in G.nodes(data=True) if attr.get('language') == 'en']
        de_nodes = [n for n, attr in G.nodes(data=True) if attr.get('language') == 'de']
        root_nodes = [en_root, f"DE:{de_root}"]
        
        # Create node positions and labels
        en_x = [pos[node][0] for node in en_nodes if node not in root_nodes]
        en_y = [pos[node][1] for node in en_nodes if node not in root_nodes]
        en_text = [node for node in en_nodes if node not in root_nodes]
        
        de_x = [pos[node][0] for node in de_nodes if node not in root_nodes]
        de_y = [pos[node][1] for node in de_nodes if node not in root_nodes]
        de_text = [G.nodes[node]['name'] for node in de_nodes if node not in root_nodes]  # Original name, not node ID
        
        root_x = [pos[node][0] for node in root_nodes]
        root_y = [pos[node][1] for node in root_nodes]
        root_text = [en_root, de_root]  # Use original German name for display
        
        # Create node traces
        node_trace_en = go.Scatter(
            x=en_x, y=en_y,
            mode='markers+text',
            text=en_text,
            textposition='top center',
            hoverinfo='text',
            marker=dict(color='skyblue', size=15, line=dict(width=1, color='black')),
            name='English Concepts'
        )
        
        node_trace_de = go.Scatter(
            x=de_x, y=de_y,
            mode='markers+text',
            text=de_text,
            textposition='top center',
            hoverinfo='text',
            marker=dict(color='lightcoral', size=15, line=dict(width=1, color='black')),
            name='German Concepts'
        )
        
        node_trace_root = go.Scatter(
            x=root_x, y=root_y,
            mode='markers+text',
            text=root_text,
            textposition='top center',
            hoverinfo='text',
            marker=dict(color='lightgreen', size=20, line=dict(width=2, color='black')),
            name='Root Concepts'
        )
        
        # Create edge traces
        edge_traces = []
        
        # English edges
        en_en_edges = [(u, v) for u, v, attr in G.edges(data=True) if attr.get('type') == 'en-en']
        for u, v in en_en_edges:
            x0, y0 = pos[u]
            x1, y1 = pos[v]
            edge_trace = go.Scatter(
                x=[x0, x1, None], y=[y0, y1, None],
                line=dict(width=1, color='rgba(0,0,255,0.3)'),
                hoverinfo='none',
                mode='lines',
                showlegend=False
            )
            edge_traces.append(edge_trace)
        
        # German edges
        de_de_edges = [(u, v) for u, v, attr in G.edges(data=True) if attr.get('type') == 'de-de']
        for u, v in de_de_edges:
            x0, y0 = pos[u]
            x1, y1 = pos[v]
            edge_trace = go.Scatter(
                x=[x0, x1, None], y=[y0, y1, None],
                line=dict(width=1, color='rgba(255,0,0,0.3)'),
                hoverinfo='none',
                mode='lines',
                showlegend=False
            )
            edge_traces.append(edge_trace)
        
        # Cross-language edges
        cross_lang_edges = [(u, v, attr) for u, v, attr in G.edges(data=True) if attr.get('type') == 'cross-lang']
        for u, v, attr in cross_lang_edges:
            x0, y0 = pos[u]
            x1, y1 = pos[v]
            weight = attr.get('weight', 0.5)
            # Adjust opacity based on weight
            opacity = min(0.8, max(0.3, weight))
            edge_trace = go.Scatter(
                x=[x0, x1, None], y=[y0, y1, None],
                line=dict(width=1.5, color=f'rgba(128,0,128,{opacity})', dash='dash'),
                hoverinfo='none',
                mode='lines',
                showlegend=False
            )
            edge_traces.append(edge_trace)
            
        # Create figure with all traces
        fig = go.Figure(data=edge_traces + [node_trace_en, node_trace_de, node_trace_root])
        
        # Update layout
        fig.update_layout(
            title=f"Interactive Cross-Language Mapping: '{en_root}' (EN) ↔ '{de_root}' (DE)",
            showlegend=True,
            hovermode='closest',
            margin=dict(b=0, l=0, r=0, t=40),
            height=800,
            width=1000,
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            plot_bgcolor='rgba(240,240,240,1)'
        )
        
        # Show the figure
        fig.show()

# Create the cross-language mapper
print("Cross-Language Semantic Mapper:")
cross_language_mapper = CrossLanguageMapper(english_graph, german_graph)
cross_language_mapper.show()

Cross-Language Semantic Mapper:


## 9. Temporal Semantic Evolution Visualizer

Let's create an interactive time-based animation that demonstrates how semantic connections form exponentially over time. This visualizer will allow us to observe the non-linear growth dynamics of semantic networks.

In [1]:
class TemporalSemanticVisualizer:
    """Interactive visualization of semantic network evolution over time with exponential growth"""
    
    def __init__(self, graph, layout=None, title="Temporal Semantic Evolution"):
        self.graph = graph
        self.title = title
        self.nodes = list(graph.nodes())
        self.edges = list(graph.edges(data=True))
        
        # Create layout or use provided one
        if layout is None:
            print("Computing network layout...")
            self.layout_pos = nx.spring_layout(graph, k=0.15, iterations=50, seed=42)
            # Convert to DataFrame
            self.layout = pd.DataFrame({
                'node': list(self.layout_pos.keys()),
                'x': [self.layout_pos[node][0] for node in self.layout_pos],
                'y': [self.layout_pos[node][1] for node in self.layout_pos]
            })
        else:
            # Convert layout to positions dict if needed
            self.layout = layout
            self.layout_pos = {row['node']: (row['x'], row['y']) 
                            for _, row in layout.iterrows()}
        
        # Setup simulation parameters
        self._setup_ui()
        self._setup_simulation()
    
    def _setup_ui(self):
        """Set up user interface for controlling the animation"""
        # Simulation speed
        self.speed_slider = widgets.FloatSlider(
            value=1.0,
            min=0.1,
            max=5.0,
            step=0.1,
            description='Speed:',
            disabled=False,
            continuous_update=True,
            orientation='horizontal',
            readout=True,
            readout_format='.1f'
        )
        
        # Network seed size
        self.seed_slider = widgets.IntSlider(
            value=5,
            min=1,
            max=50,
            step=1,
            description='Initial Seed:',
            disabled=False,
            continuous_update=True,
            orientation='horizontal',
            readout=True
        )
        
        # Growth factor
        self.growth_slider = widgets.FloatSlider(
            value=2.0,
            min=1.1,
            max=3.0,
            step=0.1,
            description='Growth Factor:',
            disabled=False,
            continuous_update=True,
            orientation='horizontal',
            readout=True,
            readout_format='.1f'
        )
        
        # Growth model 
        self.growth_model = widgets.Dropdown(
            options=['Exponential', 'Preferential Attachment', 'Small World', 'Semantic Proximity'],
            value='Exponential',
            description='Growth Model:',
            disabled=False
        )
        
        # Starting concept
        top_concepts = self._get_top_concepts(25)
        self.seed_concept = widgets.Dropdown(
            options=top_concepts,
            value=top_concepts[0] if top_concepts else None,
            description='Seed Concept:',
            disabled=False
        )
        
        # Playback controls
        self.play_button = widgets.Button(
            description='▶️ Play',
            disabled=False,
            button_style='success',
            tooltip='Start animation'
        )
        
        self.pause_button = widgets.Button(
            description='⏸️ Pause',
            disabled=True,
            button_style='warning',
            tooltip='Pause animation'
        )
        
        self.reset_button = widgets.Button(
            description='🔄 Reset',
            disabled=False,
            button_style='danger',
            tooltip='Reset animation'
        )
        
        self.save_button = widgets.Button(
            description='💾 Save GIF',
            disabled=False,
            button_style='info',
            tooltip='Save animation as GIF'
        )
        
        # Time slider
        self.time_slider = widgets.IntSlider(
            value=0,
            min=0,
            max=100,
            step=1,
            description='Time:',
            disabled=False,
            continuous_update=True,
            orientation='horizontal',
            readout=True
        )
        
        # Statistics display
        self.stats_output = widgets.Output()
        
        # Main visualization output
        self.output = widgets.Output()
        
        # Connect events
        self.play_button.on_click(self._on_play_clicked)
        self.pause_button.on_click(self._on_pause_clicked)
        self.reset_button.on_click(self._on_reset_clicked)
        self.save_button.on_click(self._on_save_clicked)
        self.time_slider.observe(self._on_time_slider_change, names='value')
        self.growth_model.observe(self._on_parameter_change, names='value')
        self.seed_concept.observe(self._on_parameter_change, names='value')
    
    def _get_top_concepts(self, n):
        """Get the top n most connected concepts in the graph"""
        degree_dict = dict(self.graph.degree())
        sorted_concepts = sorted(degree_dict.items(), key=lambda x: x[1], reverse=True)
        return [concept for concept, _ in sorted_concepts[:n]]
    
    def _setup_simulation(self):
        """Initialize simulation parameters"""
        # Simulation state
        self.is_playing = False
        self.current_step = 0
        self.max_steps = 100
        self.animation_thread = None
        self.frame_history = []
        
        # Generate growth sequence based on exponential model
        self._regenerate_growth_sequence()
    
    def _regenerate_growth_sequence(self):
        """Generate a growth sequence based on current parameters"""
        # Get parameters
        initial_seed = self.seed_slider.value
        growth_factor = self.growth_slider.value
        model = self.growth_model.value
        seed_concept = self.seed_concept.value
        
        print(f"Generating {model} growth sequence with seed={initial_seed}, factor={growth_factor}...")
        
        # Calculate number of nodes to add at each step
        if model == 'Exponential':
            # Pure exponential growth
            self.growth_sequence = [int(initial_seed * (growth_factor ** (i/10))) 
                                 for i in range(self.max_steps)]
        elif model == 'Preferential Attachment':
            # Based on Barabási–Albert model
            # Start with initial nodes, then each new node makes connections proportional to degree
            # Simulated with a slightly damped exponential
            self.growth_sequence = [int(initial_seed * (growth_factor ** (i/15) * (1 - math.exp(-i/20)))) 
                                 for i in range(self.max_steps)]
        elif model == 'Small World':
            # Based on Watts–Strogatz model
            # More linear at first, then accelerating as small-world effect kicks in
            self.growth_sequence = [int(initial_seed + (i**1.8)/2 + (growth_factor**(i/20))/2) 
                                 for i in range(self.max_steps)]
        elif model == 'Semantic Proximity':
            # Custom model that simulates how semantic networks might actually grow
            # S-curve: slow start, exponential middle, leveling off at end
            max_val = len(self.nodes)
            self.growth_sequence = [int(max_val / (1 + math.exp(-growth_factor * (i - self.max_steps/2) / 10))) 
                                 for i in range(self.max_steps)]
            # Ensure we start with at least the initial seed
            self.growth_sequence = [max(n, initial_seed) for n in self.growth_sequence]
        
        # Ensure the sequence is always increasing and capped at total nodes
        for i in range(1, len(self.growth_sequence)):
            self.growth_sequence[i] = max(self.growth_sequence[i], self.growth_sequence[i-1])
            self.growth_sequence[i] = min(self.growth_sequence[i], len(self.nodes))
        
        # Reset simulation state
        self.current_step = 0
        self.time_slider.max = self.max_steps - 1
        self.time_slider.value = 0
        
        # Generate node sequence based on seed concept
        self._generate_node_sequence(seed_concept)
    
    def _generate_node_sequence(self, seed_concept):
        """Generate a sequence of nodes to add based on the seed concept"""
        if not seed_concept or seed_concept not in self.graph:
            # If no valid seed, sort nodes by degree (highly connected nodes first)
            node_degrees = dict(self.graph.degree())
            self.node_sequence = sorted(self.nodes, key=lambda n: node_degrees.get(n, 0), reverse=True)
            return
            
        # Start with the seed concept
        visited = {seed_concept}
        sequence = [seed_concept]
        queue = [(seed_concept, 0)]  # (node, distance from seed)
        
        # Use BFS to get nodes in order of distance from seed
        while queue and len(sequence) < len(self.nodes):
            node, distance = queue.pop(0)
            
            # Get all neighbors
            neighbors = set(self.graph.neighbors(node)) | set(self.graph.predecessors(node))
            
            # Sort neighbors by weight and degree
            neighbor_scores = []
            for neighbor in neighbors:
                if neighbor in visited:
                    continue
                    
                # Calculate score based on edge weight and node degree
                weight = 0
                if self.graph.has_edge(node, neighbor):
                    weight = self.graph.get_edge_data(node, neighbor).get('weight', 1.0)
                elif self.graph.has_edge(neighbor, node):
                    weight = self.graph.get_edge_data(neighbor, node).get('weight', 1.0)
                    
                degree = self.graph.degree(neighbor)
                score = weight * (degree ** 0.5)  # Weight higher than degree
                neighbor_scores.append((neighbor, score))
            
            # Add sorted neighbors to sequence and queue
            sorted_neighbors = [n for n, _ in sorted(neighbor_scores, key=lambda x: x[1], reverse=True)]
            for neighbor in sorted_neighbors:
                if neighbor not in visited:
                    visited.add(neighbor)
                    sequence.append(neighbor)
                    queue.append((neighbor, distance + 1))
        
        # Add any remaining nodes (disconnected components)
        for node in self.nodes:
            if node not in visited:
                sequence.append(node)
                visited.add(node)
        
        self.node_sequence = sequence
    
    def _on_parameter_change(self, change):
        """Handle parameter changes"""
        # Regenerate the growth sequence with new parameters
        self._regenerate_growth_sequence()
        
        # Update the visualization
        with self.output:
            clear_output(wait=True)
            self._render_frame(self.current_step)
    
    def _on_play_clicked(self, b):
        """Handle play button click"""
        if self.is_playing:
            return
            
        self.is_playing = True
        self.play_button.disabled = True
        self.pause_button.disabled = False
        
        # Start animation in a separate thread
        import threading
        self.animation_thread = threading.Thread(target=self._animate)
        self.animation_thread.daemon = True
        self.animation_thread.start()
    
    def _on_pause_clicked(self, b):
        """Handle pause button click"""
        self.is_playing = False
        self.play_button.disabled = False
        self.pause_button.disabled = True
    
    def _on_reset_clicked(self, b):
        """Handle reset button click"""
        self.is_playing = False
        self.play_button.disabled = False
        self.pause_button.disabled = True
        self.current_step = 0
        self.time_slider.value = 0
        self.frame_history = []
        
        # Regenerate with current parameters
        self._regenerate_growth_sequence()
        
        # Update visualization
        with self.output:
            clear_output(wait=True)
            self._render_frame(0)
    
    def _on_time_slider_change(self, change):
        """Handle time slider change"""
        self.current_step = change['new']
        
        # Update visualization
        with self.output:
            clear_output(wait=True)
            self._render_frame(self.current_step)
    
    def _on_save_clicked(self, b):
        """Save the animation as a GIF"""
        import tempfile
        import os
        import uuid
        from matplotlib.animation import FuncAnimation
        import matplotlib.animation as animation
        
        with self.output:
            clear_output(wait=True)
            print("Preparing to save animation as GIF...")
            
            # Generate frames for the GIF
            if not self.frame_history or len(self.frame_history) < 10:
                print("Generating frames for the animation...")
                self.frame_history = []
                
                # Generate evenly spaced frames for smoother animation
                steps = min(30, self.max_steps)  # Limit to 30 frames for reasonable file size
                step_size = max(1, self.max_steps // steps)
                
                for step in range(0, self.max_steps, step_size):
                    # Create a figure for this step
                    fig, ax = plt.subplots(figsize=(10, 8))
                    self._render_static_frame(step, fig, ax)
                    self.frame_history.append((fig, ax))
                    print(f"Generated frame {len(self.frame_history)}/{steps}")
            
            # Create GIF filename
            output_dir = "../Data/Output"
            os.makedirs(output_dir, exist_ok=True)
            gif_filename = os.path.join(output_dir, f"semantic_evolution_{uuid.uuid4()}.gif")
            
            # Create animation
            print(f"Creating animation with {len(self.frame_history)} frames...")
            
            # Save frames as GIF
            frames = [fig for fig, _ in self.frame_history]
            
            # Set up animation writer
            writer = animation.PillowWriter(fps=2)
            
            # Create an animation using the first figure
            fig, ax = self.frame_history[0]
            ani = FuncAnimation(fig, self._update_animation_frame, frames=len(frames), interval=500)
            
            # Save the animation
            ani.save(gif_filename, writer=writer)
            
            print(f"Animation saved as: {gif_filename}")
            
            # Display the animation
            from IPython.display import Image
            display(Image(filename=gif_filename))
    
    def _update_animation_frame(self, frame_num):
        """Update function for FuncAnimation"""
        if frame_num < len(self.frame_history):
            fig, ax = self.frame_history[frame_num]
            # Clear the previous figure content
            for artist in ax.get_children():
                if not isinstance(artist, plt.Text):
                    artist.remove()
            
            # Draw the current frame
            self._render_static_frame(frame_num * (self.max_steps // len(self.frame_history)), fig, ax)
            
            return ax.get_children()
        
    def _animate(self):
        """Animation loop"""
        while self.is_playing and self.current_step < self.max_steps - 1:
            # Set the time slider, which will trigger the frame render
            self.time_slider.value = self.current_step + 1
            
            # Sleep based on speed slider
            import time
            sleep_time = 1.0 / self.speed_slider.value
            time.sleep(sleep_time)
        
        # If we reached the end, disable play and enable pause
        if self.current_step >= self.max_steps - 1:
            self.is_playing = False
            self.play_button.disabled = False
            self.pause_button.disabled = True
    
    def _render_frame(self, step):
        """Render a single frame of the animation"""
        # Create interactive or static frame based on number of nodes/edges
        num_nodes = self.growth_sequence[step]
        if num_nodes < 500:  # Use interactive for smaller graphs
            self._render_interactive_frame(step)
        else:  # Use static for larger graphs
            fig, ax = plt.subplots(figsize=(12, 10))
            self._render_static_frame(step, fig, ax)
            plt.show()
        
        # Update statistics
        with self.stats_output:
            clear_output(wait=True)
            self._display_statistics(step)
    
    def _render_interactive_frame(self, step):
        """Render an interactive frame using Plotly"""
        # Get current number of nodes to display
        num_nodes = self.growth_sequence[step]
        displayed_nodes = self.node_sequence[:num_nodes]
        
        # Create a subgraph with only these nodes
        subgraph = self.graph.subgraph(displayed_nodes)
        
        # Extract positions for nodes
        node_positions = {node: self.layout_pos[node] for node in displayed_nodes if node in self.layout_pos}
        
        # Create node trace
        node_degrees = dict(subgraph.degree())
        node_x = [pos[0] for node, pos in node_positions.items()]
        node_y = [pos[1] for node, pos in node_positions.items()]
        node_texts = list(node_positions.keys())
        node_sizes = [node_degrees.get(node, 1) * 10 for node in node_positions.keys()]
        
        node_trace = go.Scatter(
            x=node_x,
            y=node_y,
            mode='markers+text',
            text=node_texts,
            textposition='top center',
            hoverinfo='text',
            marker=dict(
                size=node_sizes,
                color=node_sizes,
                colorscale='Viridis',
                line=dict(width=1, color='black'),
                colorbar=dict(title='Connections')
            )
        )
        
        # Create edge traces
        edge_traces = []
        for u, v, data in subgraph.edges(data=True):
            if u in node_positions and v in node_positions:
                x0, y0 = node_positions[u]
                x1, y1 = node_positions[v]
                weight = data.get('weight', 1.0)
                
                edge_trace = go.Scatter(
                    x=[x0, x1, None],
                    y=[y0, y1, None],
                    line=dict(
                        width=weight * 1.5,
                        color='rgba(150,150,150,0.5)'
                    ),
                    hoverinfo='none',
                    mode='lines'
                )
                edge_traces.append(edge_trace)
        
        # Create figure
        fig = go.Figure(
            data=edge_traces + [node_trace],
            layout=go.Layout(
                title=f"{self.title} - Step {step}/{self.max_steps-1} ({num_nodes} nodes)",
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20, l=5, r=5, t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                plot_bgcolor='rgba(240,240,240,1)'
            )
        )
        
        fig.show()
    
    def _render_static_frame(self, step, fig=None, ax=None):
        """Render a static frame using Matplotlib"""
        if fig is None or ax is None:
            fig, ax = plt.subplots(figsize=(12, 10))
            
        # Get current number of nodes to display
        num_nodes = min(self.growth_sequence[step], len(self.node_sequence))
        displayed_nodes = self.node_sequence[:num_nodes]
        
        # Create a subgraph with only these nodes
        subgraph = self.graph.subgraph(displayed_nodes)
        
        # Extract positions for these nodes
        pos = {node: self.layout_pos[node] for node in displayed_nodes if node in self.layout_pos}
        
        # Calculate node sizes based on degree
        node_degrees = dict(subgraph.degree())
        sizes = [50 + 10 * node_degrees.get(node, 0) for node in pos.keys()]
        
        # Calculate edge weights
        edge_weights = [data.get('weight', 1.0) for _, _, data in subgraph.edges(data=True)]
        max_weight = max(edge_weights) if edge_weights else 1.0
        normalized_weights = [w / max_weight for w in edge_weights]
        
        # Draw the network
        nodes = nx.draw_networkx_nodes(
            subgraph, pos,
            node_size=sizes,
            node_color=list(node_degrees.values()),
            cmap=plt.cm.viridis,
            alpha=0.8,
            ax=ax
        )
        
        nx.draw_networkx_edges(
            subgraph, pos,
            alpha=0.4,
            width=[w * 2 for w in normalized_weights],
            edge_color='gray',
            ax=ax
        )
        
        # Only draw labels for high-degree nodes to reduce clutter
        threshold = np.percentile(list(node_degrees.values()), 75) if node_degrees else 0
        labels = {node: node for node in subgraph if node_degrees.get(node, 0) >= threshold}
        nx.draw_networkx_labels(
            subgraph, pos,
            labels=labels,
            font_size=8,
            font_weight='bold',
            ax=ax
        )
        
        # Add colorbar for node degrees
        if nodes is not None:  # Ensure we have nodes to create a colorbar for
            plt.colorbar(nodes, ax=ax, label='Node Connections')
        
        # Add title
        model = self.growth_model.value
        growth_factor = self.growth_slider.value
        ax.set_title(f"{self.title} - Step {step}/{self.max_steps-1}\n{model} Growth (factor={growth_factor:.1f})\n{num_nodes} nodes, {len(subgraph.edges())} connections")
        
        # Add network statistics
        if num_nodes > 1:
            try:
                density = nx.density(subgraph)
                stats_text = f"Network Density: {density:.4f}"
                if num_nodes < 1000:  # Skip these expensive calculations for large graphs
                    try:
                        avg_shortest_path = nx.average_shortest_path_length(subgraph)
                        stats_text += f"\nAvg. Path Length: {avg_shortest_path:.2f}"
                    except nx.NetworkXError:
                        # Graph might not be connected
                        stats_text += "\nAvg. Path Length: N/A (disconnected)"
                ax.text(0.02, 0.02, stats_text, transform=ax.transAxes, fontsize=10,
                      bbox=dict(facecolor='white', alpha=0.7))
            except Exception as e:
                print(f"Error calculating graph statistics: {e}")
        
        ax.axis('off')
        fig.tight_layout()
        
        return fig, ax
    
    def _display_statistics(self, step):
        """Display network growth statistics"""
        num_nodes = self.growth_sequence[step]
        displayed_nodes = self.node_sequence[:num_nodes]
        
        # Create a subgraph with only these nodes
        subgraph = self.graph.subgraph(displayed_nodes)
        
        print(f"--- Network Statistics at Step {step} ---")
        print(f"Growth Model: {self.growth_model.value}")
        print(f"Growth Factor: {self.growth_slider.value:.1f}")
        print(f"Nodes: {num_nodes}")
        print(f"Connections: {len(subgraph.edges())}")
        print(f"Avg. Connections per Node: {2*len(subgraph.edges())/num_nodes:.2f}")
        
        # Compute network density
        if num_nodes > 1:
            density = nx.density(subgraph)
            print(f"Network Density: {density:.6f}")
            
            # Only compute these for smaller networks (performance reasons)
            if num_nodes < 500:
                try:
                    # Get largest connected component
                    largest_cc = max(nx.connected_components(subgraph.to_undirected()), key=len)
                    largest_cc_size = len(largest_cc)
                    print(f"Largest Connected Component: {largest_cc_size} nodes ({largest_cc_size/num_nodes:.1%} of network)")
                    
                    # Calculate average path length in largest component
                    cc_subgraph = subgraph.subgraph(largest_cc)
                    if len(cc_subgraph) > 1:
                        avg_path = nx.average_shortest_path_length(cc_subgraph)
                        print(f"Average Path Length: {avg_path:.2f}")
                except Exception as e:
                    print(f"Error calculating advanced statistics: {e}")
        
        # Display exponential growth rate
        if step > 0:
            growth_rate = self.growth_sequence[step] / self.growth_sequence[step-1]
            print(f"Growth Rate (from previous step): {growth_rate:.2f}x")
            
            # Calculate projected network size
            if step < self.max_steps - 1:
                projected_size = int(num_nodes * growth_rate)
                print(f"Projected Next Step Size: {projected_size} nodes")
                
                # Show if projection exceeds actual graph size
                if projected_size > len(self.nodes):
                    print(f"Note: Growth will be capped at {len(self.nodes)} nodes (full network)")
    
    def show(self):
        """Display the interactive evolution visualizer"""
        # Create control panel
        parameters = widgets.VBox([
            widgets.HBox([self.seed_concept, self.seed_slider]),
            widgets.HBox([self.growth_model, self.growth_slider]),
            widgets.HBox([self.speed_slider, self.time_slider])
        ])
        
        controls = widgets.HBox([
            self.play_button,
            self.pause_button,
            self.reset_button,
            self.save_button
        ])
        
        # Main layout
        layout = widgets.VBox([
            parameters,
            controls,
            self.stats_output,
            self.output
        ])
        
        display(layout)
        
        # Render initial frame
        with self.output:
            self._render_frame(0)
        
        # Show initial statistics
        with self.stats_output:
            self._display_statistics(0)

# Create and display the temporal visualizer for English network
temporal_vis = TemporalSemanticVisualizer(english_graph, english_layout, title="English Semantic Network Evolution")
temporal_vis.show()

NameError: name 'english_graph' is not defined

## 8. Future Enhancement Opportunities

This interactive visualization system can be enhanced in several ways:

1. **3D Visualization**: Extend to 3D space for even more immersive exploration
2. **Real-time Cross-Language Mapping**: Add real-time translation and semantic mapping
3. **Semantic Search**: Implement GPU-accelerated semantic search capabilities
4. **Dynamic Graph Updates**: Support streaming updates to the semantic graph
5. **Community Detection**: Add GPU-accelerated community detection algorithms
6. **Path Analysis Improvements**: Enhance the semantic path finder with:
   - Support for bidirectional search
   - Path ranking by semantic relevance
   - Cross-language path finding
7. **Embedded Vector Visualization**: Integrate with embedding models (Word2Vec, BERT, etc.)
8. **Animation**: Create dynamic animations showing semantic evolution
9. **Custom Rendering**: Implement custom WebGL or Three.js rendering for even better performance
10. **Scalability Improvements**: Optimize for handling millions of semantic relationships